In [14]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [15]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [16]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""
FILTER_SYSTEM_PROMPT="""
You will be provide with a use case and a list of test scenario.
Based on information in the use case flow, define what test scenario is necessary to test the use case and remove duplicate test scenarios.
Remove test case test system load error that not be mentioned in use case such as: Load Failure, System Error,Network Error,Non-Existent ...(and use case not mentioned these flow)
Make sure test scenarios filted cover all the flow of use case and every scenarios in the response is unique.
Return scenario name only.
"""


In [17]:
usecase_name = "Buy product"
project_name = "ShopeeWeb"
main_flow_prompt = """
Main flow:
Step 1: User choose what product they want to buy by clicking the tick box at the head of the product's block. 
Step 2: User clicks the buy button at the right bottom of the screen.
Step 3: System redirects user to the payment page. There should be the default deliver location already chosen for the user in the delivery location section, the information section of the product including name, options, quantity, price, which shop sells it, promotion information, the shipping information section, the platform's promotion information section and the payment section with the default payment method.
Step 4: User clicks the order button.
Step 5: System removes the product from the shopping cart and forms a new order of that product.
"""
prompt_all = """
Main flow:
Step 1: User choose what product they want to buy by clicking the tick box at the head of the product's block. 
Step 2: User clicks the buy button at the right bottom of the screen.
Step 3: System redirects user to the payment page. There should be the default deliver location already chosen for the user in the delivery location section, the information section of the product including name, options, quantity, price, which shop sells it, promotion information, the shipping information section, the platform's promotion information section and the payment section with the default payment method.
Step 4: User clicks the order button.
Step 5: System removes the product from the shopping cart and forms a new order of that product.

Alternative flow 1: User choose what store they want to buy from
At step 1 of the basic flow: In case all the products they want to buy come from one store, user choose by clicking the tick box at the head of the store.
Step 2: System automatically ticks all the product comes from that store in the user's shopping cart.
Go back to step 2 in the basic flow and continue with the steps from step 2.

Alternative flow 2: User change delivery location to one of the saved location
At step 4 of the basic flow: User clicks on Change title in the delivery location section.
Step 5: User choose other saved location options.
Step 6: System changes the location information in the delivery location section corresponding to what user have chosen.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 3: User change delivery location to a new location
At step 4 of the basic flow: User clicks on Change title in the delivery location section.
Step 5: User clicks Add new location button.
Step 6: User fills in name, a correct name contains only letter and cannot be less than 2 letters. If user fills in wrong format, a notification will appears under the incorrect field and user cannot add location.
Step 7: User fills in phone number, a correct phone number must be in format (+84) xxx xxx xxx with all the x is number. If user fills in wrong format, a notification will appears under the incorrect field and user cannot add location.
Step 8: User chooses city by clicking one of the cities provided by the platform.
Step 9: User chooses district by clicking one of the districts provided by the platform.
Step 10: User chooses ward by clicking one of the wards provided by the platform.
Step 11: User fills in the exact location's address. A correct address must be an address available on the map provided by platform, if it cannot be found in the map, a notification will appears under the incorrect field and user cannot add location.
Step 12: User choose between two options if the location is their own house or their working place.
Step 13: User clicks on Finish button to save the location.
Step 14: System changes the location information in the delivery location section corresponding to what user have chosen.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 4: User adds shop or product's promotion
At step 4 of the basic flow: User clicks on Choose voucher in the product's information section.
Step 5: System shows a list of available vouchers.
Step 6: User chooses one of the available vouchers by clicking on it.
Step 7: System applies the voucher and recalculate the product's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 5: User adds platform's promotion
At step 4 of the basic flow: User clicks on Choose voucher in the platform's promotion information section.
Step 5: System shows a list of available vouchers.
Step 6: User chooses one of the available vouchers by clicking on it.
Step 7: System applies the voucher and recalculate the order's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 6: User changes shipping option
At step 4 of the basic flow: User clicks on Change title in the shipping information section.
Step 5: System shows a list of available shipping options.
Step 6: User chooses one of the available shipping options by clicking on it.
Step 7: System changes shipping method and recalculate the order's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 7: User changes payment method
At step 4 of the basic flow: User clicks on Change title in the payment method section.
Step 5: System shows a list of available payment methods.
Step 6: User chooses one of the available payment methods by clicking on it.
Step 7: System changes payment method.
Go back to step 4 in the basic flow and continue with the steps from step 4.
"""
alt_prompt=[
"""Alternative flow 1: User choose what store they want to buy from
At step 1 of the basic flow: In case all the products they want to buy come from one store, user choose by clicking the tick box at the head of the store.
Step 2: System automatically ticks all the product comes from that store in the user's shopping cart.
Go back to step 2 in the basic flow and continue with the steps from step 2.""",
"""Alternative flow 2: User change delivery location to one of the saved location
At step 4 of the basic flow: User clicks on Change title in the delivery location section.
Step 5: User choose other saved location options.
Step 6: System changes the location information in the delivery location section corresponding to what user have chosen.
Go back to step 4 in the basic flow and continue with the steps from step 4.""",
"""Alternative flow 3: User change delivery location to a new location
At step 4 of the basic flow: User clicks on Change title in the delivery location section.
Step 5: User clicks Add new location button.
Step 6: User fills in name, a correct name contains only letter and cannot be less than 2 letters. If user fills in wrong format, a notification will appears under the incorrect field and user cannot add location.
Step 7: User fills in phone number, a correct phone number must be in format (+84) xxx xxx xxx with all the x is number. If user fills in wrong format, a notification will appears under the incorrect field and user cannot add location.
Step 8: User chooses city by clicking one of the cities provided by the platform.
Step 9: User chooses district by clicking one of the districts provided by the platform.
Step 10: User chooses ward by clicking one of the wards provided by the platform.
Step 11: User fills in the exact location's address. A correct address must be an address available on the map provided by platform, if it cannot be found in the map, a notification will appears under the incorrect field and user cannot add location.
Step 12: User choose between two options if the location is their own house or their working place.
Step 13: User clicks on Finish button to save the location.
Step 14: System changes the location information in the delivery location section corresponding to what user have chosen.
Go back to step 4 in the basic flow and continue with the steps from step 4.
""",
"""Alternative flow 4: User adds shop or product's promotion
At step 4 of the basic flow: User clicks on Choose voucher in the product's information section.
Step 5: System shows a list of available vouchers.
Step 6: User chooses one of the available vouchers by clicking on it.
Step 7: System applies the voucher and recalculate the product's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.""",
"""Alternative flow 5: User adds platform's promotion
At step 4 of the basic flow: User clicks on Choose voucher in the platform's promotion information section.
Step 5: System shows a list of available vouchers.
Step 6: User chooses one of the available vouchers by clicking on it.
Step 7: System applies the voucher and recalculate the order's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.""",
"""Alternative flow 6: User changes shipping option
At step 4 of the basic flow: User clicks on Change title in the shipping information section.
Step 5: System shows a list of available shipping options.
Step 6: User chooses one of the available shipping options by clicking on it.
Step 7: System changes shipping method and recalculate the order's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.
""","""Alternative flow 7: User changes payment method
At step 4 of the basic flow: User clicks on Change title in the payment method section.
Step 5: System shows a list of available payment methods.
Step 6: User chooses one of the available payment methods by clicking on it.
Step 7: System changes payment method.
Go back to step 4 in the basic flow and continue with the steps from step 4.
"""]
exc_prompt = []

In [18]:
for i in range(1,4):
    print(f"D:\\Dissertation-GPT\\evaluate\\ResultSet\\5.20\\old filter\\{project_name}\\{usecase_name}-{i}.txt")

D:\Dissertation-GPT\evaluate\ResultSet\5.20\old filter\ShopeeWeb\Buy product-1.txt
D:\Dissertation-GPT\evaluate\ResultSet\5.20\old filter\ShopeeWeb\Buy product-2.txt
D:\Dissertation-GPT\evaluate\ResultSet\5.20\old filter\ShopeeWeb\Buy product-3.txt


In [20]:

for i in range(1,4):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")
  for alt in alt_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
      ]
    sub_gpt_response = ask(promptSubScenario, client, model)
    print(alt)
    print("\nsub_gpt_response:"+sub_gpt_response +"\n")
    filtercontent+=  "\n" + sub_gpt_response
  for exc in exc_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
      ]
    sub_gpt_response = ask(promptSubScenario, client, model)
    print(exc)
    print("\nsub_gpt_response:"+sub_gpt_response +"\n")
    filtercontent+=  "\n" + sub_gpt_response
    sub_gpt_response = ask(promptSubScenario, client, model)


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"D:\\Dissertation-GPT\\evaluate\\ResultSet\\5.23\\run propose\\{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 2
{'Name field': {'condition': {'valid': 'name contains only letters and is not less than 2 letters', 'invalid': 'name contains characters other than letters or is less than 2 letters'}, 'type': 'text field'}, 'Phone number field': {'condition': {'valid': 'phone number is in format (+84) xxx xxx xxx with all x being numbers', 'invalid': 'phone number is not in the specified format or contains characters other than numbers'}, 'type': 'text field'}, "Exact location's address field": {'condition': {'valid': 'address is available on the map provided by platform', 'invalid': 'address cannot be found on the map'}, 'type': 'text field'}}
1. Scenario: Add new location with name containing numbers
2. Scenario: Add new location with name less than 2 letters
3. Scenario: Add new location with phone number in incorrect format
4. Scenario: Add new location with phone number containing letters
5. Scenario: Add new location with address not available on the platform's map

Main flow:
Step 1: Use